In [1]:
import cvxpy as cp

In [2]:
import numpy as np

In [7]:
def maxmin_sinr(G, P_max, P_received, sigma, Group, Group_max, epsilon = 0.001):
    # Find G matrix dimension
    n, m = np.shape(G) # n = rows, m = column

    
    if m != np.size(P_max):
        print('Error\n')
        return 'Error\n', np.nan, np.nan, np.nan

    if n != np.size(P_received):
        print('Error\n')
        return 'Error', np.nan, np.nan, np.nan

    if n != np.size(sigma):
        print('Error: σ dimensions do not match gain matrix dimensions\n')
        return 'Error: σ dimensions do not match gain matrix dimensions', np.nan, np.nan, np.nan

    

    delta = np.identity(n) #delta is identity matrix of nxn
    S = G*delta # signal_power matrix
    I = G-S # interference_power matrix

    # group matrix: number of groups x number of transmitters
    num_groups = int(np.size(Group,0))

    if num_groups != np.size(Group_max):
        print('Error: Number of groups from Group matrix does not match dimensions of Group_max\n')
        return ('Error: Number of groups from Group matrix does not match dimensions of Group_max',
                np.nan, np.nan, np.nan, np.nan)

    # normalising
    Group_norm = Group/np.sum(Group,axis=1).reshape((num_groups,1))


    p = cp.Variable(shape=n) #p: the power of the n transmitters
    best = np.zeros(n)

    # set upper and lower bounds for sub-level set
    u = 1e4 #set upper for sub-level set
    l = 0 #set lower for sub-level set

    
    alpha = cp.Parameter(shape=1)

    # bisection feasibility test constraints
    constraints = [I*p + sigma <= alpha*S*p, p <= P_max, p >= 0, G*p <= P_received, Group_norm*p <= Group_max]

    # objective function definition
    obj = cp.Minimize(alpha)

    # Check: solution lies between u and l
    alpha.value = [u]
    prob = cp.Problem(obj, constraints)
    prob.solve()

    if prob.status != 'optimal':
        #level set u is below the solution
        print('There are no any optimal solution present within bounds\n')
        return 'Error: There are no any optimal solution present within bounds', np.nan, np.nan, np.nan

    alpha.value = [l]
    prob = cp.Problem(obj, constraints)
    prob.solve()

    if prob.status == 'optimal':
        # level set l is below the solution
        print('There are no optimal solution present within bounds\n')
        return 'Error: There are no optimal solution present within bounds', np.nan, np.nan, np.nan

    # Bisection algortithm
    maxLoop = int(1e7)
    for i in range(1,maxLoop):
        
        alpha.value = np.atleast_1d((u + l)/2.0)

        
        if u-l <= epsilon:
            break

        # form the
        prob = cp.Problem(obj, constraints)
        #solve problem
        prob.solve()

        
        if prob.status == 'optimal':
            u = alpha.value
            best = p.value
        else:
            l = alpha.value

        
        if u - l > epsilon and i == (maxLoop-1):
            print("Solution not converged to order epsilon")

    return l, u, float(alpha.value), best

In [8]:
np.set_printoptions(precision=3)

#Signal weight = 0.8 and Interference weight = 0.1
G = np.array([[0.8,0.1,0.1,0.1,0.1],
              [0.1,0.8,0.1,0.1,0.1],
              [0.1,0.1,0.8,0.1,0.1],
              [0.1,0.1,0.1,0.8,0.1],
              [0.1,0.1,0.1,0.1,0.8]])


n, m = np.shape(G) # Here we are considering n = m

P_max = np.array([1.]*n)


P_received = np.array([4.,4.,4.,4.,4.])/n


sigma = np.array([0.1,0.1,0.1,0.1,0.1])


Group = np.array([[1.,1.,0,0,0],[0,0,1.,1.,1.]])


Group_max = np.array([1.8,1.8])


l, u, alpha, best = maxmin_sinr(G, P_max, P_received, sigma, Group, Group_max)

print('Minimum SINR={:.4g}'.format(1/alpha))
print('Power={}'.format(best))

Minimum SINR=1.454
Power=[0.666 0.666 0.666 0.666 0.666]


C:\Users\SWARAJ\anaconda3\lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 9 times so far.

  warnings.warn(msg, UserWarning)
C:\Users\SWARAJ\anaconda3\lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 10 times so far.

  warnings.warn(msg, UserWarning)
C:\Users\SWARAJ\a